In [1]:
import numpy as np
import pandas as pd
from sklearn.utils import shuffle
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from keras.utils import to_categorical
from keras.models import Sequential
from keras.layers import Dense

In [3]:
in_data = pd.read_csv('./Dataset/BP_Dataset.csv')
in_data.head()

,Age(year),Gender,Height(cm),Weight(kg),BMI(kg/m^2),Systolic Blood Pressure(mmHg),Diastolic Blood Pressure(mmHg),BP,cholesterol,gluc,smoke,alco,active,Target
0,50,2,168,62.0,21.967120,110,80,90.000000,1,1,0,0,1,Normal
1,55,1,156,85.0,34.927679,140,90,106.666667,3,1,0,0,1,Stage2
2,51,1,165,64.0,23.507805,130,70,90.000000,3,1,0,0,0,Stage1
3,48,2,169,82.0,28.710479,150,100,116.666667,1,1,0,0,1,Stage1
4,47,1,156,56.0,23.011177,100,60,73.333333,1,1,0,0,0,Normal


In [5]:
in_data = shuffle(in_data)
X = in_data.drop(['Target'], axis = 1)
X = np.array(X)
Y = np.array(in_data['Target'])
X[:10], Y[:10]

(array([[ 60.        ,   2.        , 170.        ,  55.        ,
          19.03114187, 140.        ,  80.        ,  99.9999998 ,
           1.        ,   1.        ,   0.        ,   0.        ,
           1.        ],
        [ 44.        ,   1.        , 162.        ,  62.        ,
          23.62444749, 130.        ,  80.        ,  96.6666665 ,
           1.        ,   1.        ,   0.        ,   0.        ,
           0.        ],
        [ 54.        ,   2.        , 167.        ,  65.        ,
          23.30668005, 120.        ,  80.        ,  93.3333332 ,
           1.        ,   1.        ,   0.        ,   0.        ,
           0.        ],
        [ 53.        ,   2.        , 160.        ,  81.        ,
          31.640625  , 140.        ,  80.        ,  99.9999998 ,
           1.        ,   2.        ,   0.        ,   0.        ,
           1.        ],
        [ 56.        ,   1.        , 174.        ,  69.        ,
          22.79032897, 120.        ,  80.        ,  93.3333

In [16]:
l_encode = LabelEncoder()
l_encode.fit(Y)
Y = l_encode.transform(Y)
Y = to_categorical(Y)
Y

array([[0., 0., 1., 0.],
       [0., 1., 0., 0.],
       [0., 1., 0., 0.],
       ...,
       [0., 0., 1., 0.],
       [0., 0., 1., 0.],
       [0., 1., 0., 0.]], dtype=float32)

In [17]:
train_x, test_x, train_y, test_y = train_test_split(X,Y, test_size = 0.3, random_state = 0)
train_x.shape, train_y.shape, test_x.shape, test_y.shape

((39200, 13), (39200, 4), (16800, 13), (16800, 4))

In [20]:
in_dim = len(in_data.columns)-1

model = Sequential()
model.add(Dense(8, input_dim = in_dim, activation = 'relu'))
model.add(Dense(10, activation = 'relu'))
model.add(Dense(10, activation = 'relu'))
model.add(Dense(10, activation = 'relu'))
model.add(Dense(4, activation = 'softmax'))

model.compile(loss = 'categorical_crossentropy', optimizer = 'adam', metrics = ['accuracy'])
model.fit(train_x, train_y, epochs = 15, batch_size = 5)
scores = model.evaluate(test_x, test_y)

for i, m in enumerate(model.metrics_names):
    print("\n%s: %.3f"% (m, scores[i]))

Epoch 1/15
7840/7840 [==============================] - 9s 1ms/step - loss: 0.5284 - accuracy: 0.8362
Epoch 2/15
7840/7840 [==============================] - 8s 1ms/step - loss: 0.2355 - accuracy: 0.9189
Epoch 3/15
7840/7840 [==============================] - 8s 1ms/step - loss: 0.1192 - accuracy: 0.9608
Epoch 4/15
7840/7840 [==============================] - 8s 1ms/step - loss: 0.0736 - accuracy: 0.9772
Epoch 5/15
7840/7840 [==============================] - 9s 1ms/step - loss: 0.0641 - accuracy: 0.9807
Epoch 6/15
7840/7840 [==============================] - 9s 1ms/step - loss: 0.0541 - accuracy: 0.9841
Epoch 7/15
7840/7840 [==============================] - 9s 1ms/step - loss: 0.0570 - accuracy: 0.9833
Epoch 8/15
7840/7840 [==============================] - 8s 1ms/step - loss: 0.0491 - accuracy: 0.9850
Epoch 9/15
7840/7840 [==============================] - 8s 1ms/step - loss: 0.0519 - accuracy: 0.9845
Epoch 10/15
7840/7840 [==============================] - 8s 1ms/step - loss: 0.047

In [21]:
test_size = 10
pred_probs = model.predict(test_x[:test_size])
pred = np.argmax(pred_probs, axis=1)
pred_ = l_encode.inverse_transform(pred)

true_y = l_encode.inverse_transform(np.argmax(test_y[:test_size], axis=1))

for i, j in zip(pred_, true_y):
    print("Predicted: {}, True: {}".format(i, j))


1/1 [==============================] - 0s 57ms/step
Predicted: Stage1, True: Stage1
Predicted: Elevated, True: Elevated
Predicted: Stage1, True: Stage1
Predicted: Stage1, True: Stage1
Predicted: Normal, True: Normal
Predicted: Normal, True: Normal
Predicted: Stage1, True: Stage1
Predicted: Stage2, True: Stage2
Predicted: Elevated, True: Elevated
Predicted: Normal, True: Normal


In [22]:
# Save the trained model weights to a file
model.save('new_model.h5')

c:\Users\hp\AppData\Local\Programs\Python\Python311\Lib\site-packages\keras\src\engine\training.py:3079: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


In [26]:
import joblib

# Save the LabelEncoder's classes_ attribute to a file
joblib.dump(l_encode.classes_, 'label_encoder_classes.pkl')



['label_encoder_classes.pkl']